https://use-the-index-luke.com/sql/where-clause/searching-for-ranges/like-performance-tuning

This notebook is because i want to test the speed difference between a full text index vs a normal index

In [5]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

init_db(2_000_000)

Created empty database
Created empty tables
Starting to insert data

Starting insert into 'products' (2000000 rows)...
Finished inserting 2000000 rows into 'products' in 36.66s.


This initial is just a benchmark without any indexing 

In [9]:
drop_non_clustered_indexes("products", get_database_name())

query = """
SELECT * FROM products WHERE name like '%%Charger%%';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " +query, database=get_database_name(), print_as_df=True, show_metrics=False)

[QUERY METRICS] 88235 rows fetched, 1577.74 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1979023,11.11,Using where


In [10]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE INDEX idx_name ON products(name);
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

query = """
SELECT * FROM products WHERE name like '%%Charger%%';
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " +query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 88235 rows fetched, 1540.90 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,ALL,None,None,None,None,1979023,11.11,Using where


In [12]:
drop_non_clustered_indexes("products", get_database_name())

query = """
CREATE FULLTEXT INDEX idx_name ON products(name);
"""

execute_query(query, database=get_database_name(), print_as_df=True, show_metrics=False)

query = """
SELECT * FROM products WHERE MATCH(name) AGAINST('Charger');
"""

execute_query(query, database=get_database_name(), print_as_df=False, show_metrics=True)
execute_query("EXPLAIN " +query, database=get_database_name(), print_as_df=True, show_metrics=False)


[QUERY METRICS] 88235 rows fetched, 980.60 ms


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products,None,fulltext,idx_name,idx_name,0,const,1,100.0,Using where; Ft_hints: sorted
